# BC4 Setup

We will help you set up SSH so you can connect to [BlueCrystal 4 (BC4)](https://www.acrc.bris.ac.uk/acrc/phase4.htm) 
and ensure that you have got your environment set up correctly. We'll also discuss some of the basics of the system, 
drawing attention to aspects that will be relevant to you throughout this course.




## SSH configuration

**University laptops only:** Skip straight to [University laptop setup](#University-laptop-setup), none of the following instructions are relevant to you.

**Windows users only:**
If you're running windows, [enable WSL](https://docs.microsoft.com/en-us/windows/wsl/install-win10) and install the Ubuntu 18.04 LTS distribution from the Microsoft Store. Once installed, open up the Ubuntu application which should launch a bash shell. WSL keeps the linux filesystem separate from the Windows one, however Windows drives are mounted at `/mnt/<drive-letter>`, allowing you to transparently access files on your Windows machine from the Ubuntu subsystem.

If you are unable to install WSL, then follow the steps in [Windows setup: Non-WSL](#Windows-setup:-Non-WSL)

**The following steps should be followed by everyone apart from those borrowing university laptops**

- Open up `~/.ssh/config` in your editor and add the following lines

  ```
  # file: ~/.ssh/config
  Host snowy
      User <UoB-username>
      HostName snowy.cs.bris.ac.uk
      
  # Login in when you're outside of the University.
  Host bc4-external
      User <UoB-username>
      HostName bc4login.acrc.bris.ac.uk
      ProxyJump snowy  # This only works on SSH 7.3+
      
  # But when you're inside the University you can avoid connecting via snowy.
  Host bc4
      User <UoB-username>
      HostName bc4login.acrc.bris.ac.uk
  ```
  Then run `chmod 600 ~/.ssh/config` so it is only readable by you.
  
  **NOTE:** Make sure to replace `<UoB-username>` with your actual username, e.g. `wp14283`.
- If you don't already have an SSH key `~/.ssh/id_rsa`, then generate one with `ssh-keygen` accepting all the defaults.
- Copy you SSH public key over to snowy and BC4

  ```console
  $ ssh-copy-id snowy
$ ssh-copy-id bc4
  ```
  
  Note this will prompt you to enter your password so `ssh-copy-id` can login and copy your public key to `~/.ssh/authorized_keys` on the remote server.
- Check that `ssh bc4` logs onto BC4 without prompting you for your password.
- When you're at home or outside the university network, you can login to BC4 with `ssh bc4-external`

---

### University laptop setup

The following section is only necessary for those using a Windows laptop borrowed from University. If this doesn't apply to you, [skip to the next section](#GPU-test)

The unversity provided laptops have [PuTTY](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html), an SSH client, installed on them. We will have to configure PuTTY to connect to BC4. 

Watch the video provided below that details the process of setting up PuTTY for connection to BC4. We provide detailed instructions below.

[![PuTTY Setup video still](./media/putty-video-still.png)](https://www.youtube.com/watch?v=9leG8dfD48Q&feature=youtu.be)

The steps shown in the video are explained below:

1. Open PuTTYGen (type *puttygen* into the start menu search bar)

   ![PuTTYGen](./media/bc4-setup/1-putty-gen.png)
2. Click the *Generate* button and move you mouse of the key area.

   ![PuTTYGen generating keys](./media/bc4-setup/2-putty-gen.png)
3. Wait for you key to be generated once sufficient randomness was collected by moving your mouse

   ![PuTTYGen generated key](./media/bc4-setup/3-putty-gen.png)
4. Save your public key as `id_rsa.pub` in your documents folder.
5. Save your private key as `id_rsa.ppk` in your documents folder.
6. Open PuTTY (type *putty* into the start menu search bar)

   ![PuTTY](./media/bc4-setup/4-putty.png)
7. Type `bc4login.acrc.bris.ac.uk` into the *Host Name* box in PuTTY.
8. Click *Open*.

   ![PuTTY initial warning](./media/bc4-setup/5-putty-warning.png)
9. Click *Yes* to trust the server's host key

   ![PuTTY credentials prompt](./media/bc4-setup/6-putty-connection.png)
10. You should now see a username/password propmpt like the one above. Type in your user name (e.g. wp13824) and then enter your password. Beware your password characters *will not show up*, although the keypressed are being registered. You should then be met with a bash prompt on BC4.

   ![Creating `authorized_keys`](./media/bc4-setup/7-authorized-keys.png)
11. Create a folder `.ssh` in your BC4 home directory by running `mkdir -p .ssh`
12. Open the new file `.ssh/authorized_keys` using `nano`, a terminal text editor: type `nano .ssh/authorized_keys`

    ![Copying authorized key from PuTTYGen](./media/bc4-setup/8-authorized-key.png)
13. Copy the key you generated earlier from PuTTYGen, that's the text inside the box below the text *"Public key for pasting into OpenSSH authorized_keys file:"*.

    ![Paste the key into the authorized_key buffer of nano](./media/bc4-setup/9-authorized-key.png)
14. Paste the key into the `.ssh/authorized_key` file you have open in `nano`. Do this by **right-clicking** on the terminal window, this is how to paste in PuTTY. Save the file by pressing `<Ctrl>-x` and answering `Y` to the question of whether to save the file or not.

    ![BC4 putty session setup 1](./media/bc4-setup/10-putty-settings-1.png)
15. Open PuTTY again, set the host name to `bc4login.acrc.bris.ac.uk`, in teh *Saved Sessions* input box, type `BC4`.

    ![BC4 putty session setup 2](./media/bc4-setup/10-putty-settings-2.png)
16. Select *Connection > SSH > Auth* in the left hand menu, click *Browse* next to the input box for *Private key file for authentication*. Point it towards your `id_rsa.ppk` key. 

    ![BC4 putty session setup 3](./media/bc4-setup/10-putty-settings-3.png)
17. Select *Connection > Data* in the left hand menu, in the *Auto-login username* box, enter your university username, e.g. *wp13824*

    ![BC4 putty session setup 4](./media/bc4-setup/10-putty-settings-4.png)
18. Go back to the top level *Session* tab in the left-hand menu. Save your session by clicking the *Save* button next to the list of saved sessions.

    ![BC4 terminal login](./media/bc4-setup/11-ssh.png)
19. Check you can now login into BC4 without a password by clicking *Open*, you should see a terminal prompt like that shown above.

    ![Pageant taskbar icon](./media/bc4-setup/14-taskbar.png) 
20. Now we need to set up a SSH agent which will cache your private keys and use them to login to BC4 whenever you use SSH. Pageant is PuTTY's SSH agent. You can load your private key `id_rsa.ppk` into Pageant by double clicking on it. This will start a Pageant process if one is not already running creating an icon in the taskbar. You can see this in the above screenshot to the right of the ^ icon. 

21. You can right click on the pageant icon and launch an SSH terminal by navigating and clicking *Saved sessions > BC4*. Check to see if this works.

    ![Startup run dialog](./media/bc4-setup/startup.png)
    ![Startup run dialog](./media/bc4-setup/startup-folder.png)
22. We can automatically load Pageant everytime we login by adding a shortcut in the windows startup folder. To do this type `<Win>-r` to open the run prompt and type `shell:startup`. This will open a file explorer in the start up folder. Open up another file explorer and navigate to the folder containing `id_rsa.ppk`. Right click the `id_rsa.ppk` file and click copy. Go back to the startup folder window and right-click and *paste a shortcut*. Now every time you login that file will be executed which will cause pageant to be launched and your private key to be loaded.

    ![pscp example](./media/bc4-setup/12-pscp-test.png)
23. Copying files using PuTTY is a bit clunky, you will have to use `cmd.exe` and `pscp` instead of `scp`. For example, let's create a new file called `test.txt` with the contents `"hi"`.
    Open `cmd.exe` by typing *cmd* in the start menu seach bar. Type `echo "hi" > test.txt` to create our test.txt file. Now copy that file to BC4 by using `pscp`. The syntax of this command is `pscp <src> <user>@<host>:[<dest>]`. If `<dest>` is ommitted, then file will be copied to your home directory. Copy the `test.txt` file to your home directory by running `pscp test.txt <user>@bc4login.acrc.bris.ac.uk:`. 
    
If `pscp` asks you for your password it means pageant is not configured correctly/not running. `pscp` will query pageant for private keys when it trys to log into a server.

---
  
## GPU test

In order to make sure that 
will handle the coming lab sessions smoothly follow the steps to train and test a simple classifier below:

1. Copy the files `train.sh`  and `train_mnist.py` alongisde this notebook in the github repository (clone the repository to get the files) to your BC4 account using `scp`:

   ```console
   $ scp /path_to_files/{train.sh,train_mnist.py} bc4:
   ```
   
2. Open a terminal window and login to BC4 via ssh:

   ```console
   $ ssh bc4
   ```

3. Make the script executable (we use `[bc4] $` to denote when we are running commands in an SSH session on BC4)
   ```console
   [bc4] $ chmod +x train.sh train_mnist.py
   ```  

4. Load the software package that contains all the dependencies needed for this course

   ```console
   [bc4] $  module load languages/anaconda3/2019.07-3.6.5-tflow-1.14
   ```
   
5. Execute the script by typing 

   ```console
   [bc4] $ ./train.sh
   ``` 
   
Running the commands above should produce an output similar to that shown below:

```
9920512it [00:05, 1764739.10it/s]                            
32768it [00:00, 120496.12it/s]           
1654784it [00:00, 2184813.40it/s]                           
8192it [00:00, 46039.25it/s]
Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to data/MNIST/raw/train-images-idx3-ubyte.gz
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz
Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz
Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz to data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz
Downloading http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz to data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!

Train Epoch: 1 [0/60000 (0%)]   Loss: 2.300039
Train Epoch: 1 [640/60000 (1%)] Loss: 2.213470
Train Epoch: 1 [1280/60000 (2%)]        Loss: 2.170460
Train Epoch: 1 [1920/60000 (3%)]        Loss: 2.076698
Train Epoch: 1 [2560/60000 (4%)]        Loss: 1.868078
Train Epoch: 1 [3200/60000 (5%)]        Loss: 1.414199
Train Epoch: 1 [3840/60000 (6%)]        Loss: 1.000870
Train Epoch: 1 [4480/60000 (7%)]        Loss: 0.775734
Train Epoch: 1 [5120/60000 (9%)]        Loss: 0.460020
Train Epoch: 1 [5760/60000 (10%)]       Loss: 0.486314
Train Epoch: 1 [6400/60000 (11%)]       Loss: 0.437761
Train Epoch: 1 [7040/60000 (12%)]       Loss: 0.410439
...
```

If an error appears at any point please let us know, otherwise type `exit` on your terminal window (or press `<Ctrl> + d`) to close the BC4 SSH session.

### Windows setup: Non-WSL

If you are using a Windows install that you can't install WSL on then you will have to use an alternate SSH client. We suggest installing [cmder](https://cmder.net/) which comes with a version of openssh installed. You won't be able to use `ssh-copy-id`, and will instead have to follow these steps:

1. Generate an SSH key using `ssh-keygen` and accept all the defaults. This will generate `~/.ssh/id_rsa` and `~/.ssh/id_rsa.pub`.
2. Copy the contents of `%USERPROFILE%\.ssh\id_rsa.pub` into a notepad window.
3. Follow the set up instructions in the section above to create  `%USERPROFILE%\.ssh\config`.
4. SSH to BC4: `ssh bc4`
5. Open up nano: `nano ~/.ssh/authorized_keys` and copy and paste the contents of `~/.ssh/id_rsa.pub` that you copied into the notepad window earlier. Make sure it all resides on a single line. Write and exit the file by pressing `<Ctrl>-O` then `<Ctrl>-X`.
6. Exit the SSH session by pressing `<Ctrl>-d` or typing `exit`
7. SSH to BC4: `ssh bc4`, verify that it no longer asks you for your password, if it does, ask a TA for help.

---

## An introduction to BC4

High Performance Computing (HPC) systems like BC4 all are set up in a similar way. Typically there are machines
called *login nodes* whose sole purpose is for you to login to (via SSH) and submit jobs (there are 5 of these nodes in BC4). When you SSH onto BC4 you will be placed onto one of these nodes--they are not for running experiments! *Compute nodes* account for the majority
of the machines in a super computer (there are 32 nodes in BC4 with dual P100 GPUs), these are the machines that *are* for running experiments. 
To run something on a compute node, you must wrap it up into a *job*. A job is simply a unit of work, e.g. a script. When you submit a job it is put into a queue of jobs from which compute nodes pull work from until there is none left (except there are always more jobs than machines to complete them!). 

So how does this work in practice? Well BC4 uses the [slurm](https://slurm.schedmd.com/documentation.html) job management system. Jobs are submitted using `srun` or `sbatch` and the queue is inspected with `squeue`. 

Let's dig into this a bit deeper and practice running jobs on BC4.
We'll differentiate between shells by using the prompt `$` for a shell session on your local computer/laptop, `[bc4] $` for one on a BC4 login node, and `[bc4-compute-node] $` for one on a compute node.


First we'll log on to a login node:

```console
$ ssh bc4
[bc4] $
```

Next we'll run an interactive job to gain access to a shell on a compute node

```console
[bc4] $ srun --partition gpu --gres gpu:1 --account comsm0018 --time 0-00:15 --mem=64GB --reservation comsm0018-lab1 --pty bash
[bc4-compute-node] $
```

**NOTE: the reservation ``comsm0018-lab1``. You will be changing this every lab, given the lab number.**

**NOTE 2: when running this OUTSIDE a lab session, change into**

```console
[bc4] $ srun --partition gpu --gres gpu:1 --time 0-00:15 --mem=64GB --pty bash
[bc4-compute-node] $
```

#### We cannot guarantee interactive sessions outside assigned labs. You will learn other ways to use BC4 outside lab sessions later.

It's important not to stay on a compute node unless you are going to make use of its resources. Be polite and close your interactive sessions when you no longer need them.

```console
[bc4-compute-node] $ exit  # close the interactive session
[bc4] $
```

Software on BC4 is managed through a module system called [lmod](https://lmod.readthedocs.io/). 

We have a module specifically set up for use in this course called `languages/anaconda3/2019.07-3.6.5-tflow-1.14`, this contains 

- Python 3.7
- PyTorch 1.2.0
- torchvision 0.4.0
- matplotlib 3.1.1
- seaborn 0.9.0
- scikit-learn 0.23.1
- tensorflow 1.14 (primarily as it contains tensorboard, a metrics logging tool we shall use)

You'll need to ensure its always loaded before running your programs. We suggest you add the following line to your `~/.bashrc` file on BC4

```bash
# file: ~/.bashrc
module load languages/anaconda3/2019.07-3.6.5-tflow-1.14
```

This will ensure that every time you login to BC4 this course's module will be loaded.